In [ ]:
cd ..

In [ ]:
from pkg.util import winner_rings
from pkg.util import damage

In [ ]:

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Dropout, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve

In [ ]:
import warnings
warnings.simplefilter('ignore')

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

random_state = 47
np.random.seed(random_state)

In [ ]:
# mode = max for monitro val_accuracy and val_prc

early_stopping = EarlyStopping(
        monitor='val_loss', 
        verbose=1,
        patience=10,
        mode='min',
        restore_best_weights=True)

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
dense_16 = 0
dense_32 = 0
cnn_16 = 0
cnn_32 = 0

In [ ]:
def plot_loss(history, label, n):
    plt.semilogy(history.epoch, history.history['loss'],
                 color=colors[n], label='Train ' + label)
    plt.semilogy(history.epoch, history.history['val_loss'],
                 color=colors[n], label='Val ' + label,
                 linestyle="--")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
def plot_metrics(history):
    metrics = ['loss', 'prc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])
        plt.legend()

def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    print('True Negatives: ', cm[0][0])
    print('False Positives: ', cm[0][1])
    print('False Negatives: ', cm[1][0])
    print('True Positives: ', cm[1][1])
    print('Total: ', np.sum(cm[1]))

def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

def plot_prc(name, labels, predictions, **kwargs):
    precision, recall, _ = precision_recall_curve(labels, predictions)

    plt.plot(precision, recall, label=name, linewidth=2, **kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

# Prepare Dataset

In [ ]:
mse_url = "output/ringsranking_mse_sixteen.csv"
rings_url = "output/ringsranking_rings_sixteen.csv"
df_ringsranking_sixteen = winner_rings(mse_url, rings_url)

mse_url = "output/ringsranking_mse_thirtytwo.csv"
rings_url = "output/ringsranking_rings_thirtytwo.csv"
df_ringsranking_thirtytwo = winner_rings(mse_url, rings_url)

In [ ]:
mse_url = "output/pine_mse_sixteen.csv"
rings_url = "output/pine_rings_sixteen.csv"
df_pine_sixteen = winner_rings(mse_url, rings_url)

mse_url = "output/pine_mse_thirtytwo.csv"
rings_url = "output/pine_rings_thirtytwo.csv"
df_pine_thirtytwo = winner_rings(mse_url, rings_url)

In [ ]:
mse_url = "output/fur_mse_sixteen.csv"
rings_url = "output/fur_rings_sixteen.csv"
df_fur_sixteen = winner_rings(mse_url, rings_url)

mse_url = "output/fur_mse_thirtytwo.csv"
rings_url = "output/fur_rings_thirtytwo.csv"
df_fur_thirtytwo = winner_rings(mse_url, rings_url)

In [ ]:
mse_url = "output/tracy_mse_sixteen.csv"
rings_url = "output/tracy_rings_sixteen.csv"
df_tracy_sixteen = winner_rings(mse_url, rings_url)

mse_url = "output/tracy_mse_thirtytwo.csv"
rings_url = "output/tracy_rings_thirtytwo.csv"
df_tracy_thirtytwo = winner_rings(mse_url, rings_url)

In [ ]:
#damage
damage_url = "dataset/damage.xlsx"
df_damage_sixteen = damage(damage_url, df_ringsranking_sixteen, df_pine_sixteen, df_fur_sixteen, df_tracy_sixteen)
df_damage_thirtytwo = damage(damage_url, df_ringsranking_thirtytwo, df_pine_thirtytwo, df_fur_thirtytwo, df_tracy_thirtytwo)

# 1 Dense Model

In [ ]:
def layerFactory(type, nodes, activation):
    if type == 'Dense':
        if not activation:
            return Dense(units=nodes)
        else:
            return Dense(units=nodes, activation=activation)
    if type == 'Conv1D':
        if not activation:
            return Conv1D(units=nodes)
        else:
            return Conv1D(units=nodes, activation=activation)
    if type == 'MaxPooling1D':
        if not activation:
            return MaxPooling1D(units=nodes)
        else:
            return MaxPooling1D(units=nodes, activation=activation)
    if type == 'Dropout':
        return Dropout(rate=nodes)
    if type == 'Flatten':
        if not activation:
            return Flatten(units=nodes)
        else:
            return Flatten(units=nodes, activation=activation)
    
    return Dense(units=nodes, activation=activation)

In [ ]:
def make_dense_model(layers, input_dim, loss, optimizer, learning_rate=0.001, metrics=METRICS, output_bias=None):

    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)

    model = Sequential([
        InputLayer(input_shape=(input_dim))
    ])

    for layer in layers:
        model.add(layerFactory(layer['type'], layer['nodes'], layer['activation']))

    optimizer = tf.keras.optimizers.get(optimizer)
    optimizer.learning_rate.assign(learning_rate)

    model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=metrics)

    print(model.summary())

    return model

## 1.1 Dense Model with 16 Lines

### 1.1.1 Load Prepared Dataset

In [ ]:
#columns to drop
#columns = ['min', 'max', 'mean', 'orig', 'image', 'algo', 'Image', 'Group']
columns = ['image', 'algo', 'Image', 'Group']

df = df_damage_sixteen.drop(columns=columns)

### 1.1.2 Split into sets for training, validation, and testing + use and save scaler

In [ ]:
X = df.drop(columns=['Damage'])
y = df['Damage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=random_state)

scaler_sixteen = StandardScaler()
X_train = scaler_sixteen.fit_transform(X_train)

X_val = scaler_sixteen.transform(X_val)
X_test = scaler_sixteen.transform(X_test)

print('Training features shape:', X_train.shape)
print('Training labels shape:', y_train.shape, '\n')

print('Validation features shape:', X_val.shape)
print('Validation labels shape:', y_val.shape, '\n')

print('Test features shape:', X_test.shape)
print('Test labels shape:', y_test.shape, '\n')

### 1.1.3 Experimentation

#### 1.1.3.1 Configurations

In [ ]:
EPOCHS = 500
BATCH_SIZE = 50

input_dim = X_train.shape[1]
output_size = 1

units = 16
activation = 'relu'
#activation = 'tanh'
kernel_initializer = 'he_uniform'
output_activation = 'sigmoid'
optimizer = 'Adam'
learning_rate = 1e-3

LAYERS = [ 
    { 'type': 'Dense', 'nodes':units, 'activation': activation },
    { 'type': 'Dense', 'nodes':units, 'activation': activation },
    { 'type': 'Dense', 'nodes':output_size, 'activation': output_activation }
]

#### 1.1.3.2 Model

In [ ]:
model = make_dense_model(metrics=METRICS,
                         layers=LAYERS, 
                         loss='binary_crossentropy',
                         optimizer=optimizer,
                         learning_rate=learning_rate, 
                         input_dim=input_dim)

In [ ]:
history = model.fit(x=X_train,
                    y=y_train, 
                    validation_data=(X_val, y_val), 
                    shuffle=True, 
                    epochs=EPOCHS, 
                    verbose=1,
                    batch_size=BATCH_SIZE, 
                    callbacks=[early_stopping])

In [ ]:
early_stopping.stopped_epoch

In [ ]:
#plot_loss(history, "Bias", 0)
#plt.show()

In [ ]:
plot_metrics(history)
#plt.show()

In [ ]:
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)

# Confusion matrix
results = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
    if name == 'accuracy':
        dense_16 = value

In [ ]:
plot_cm(y_test, predictions_test)
#plt.show()

In [ ]:
plot_roc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Precision-Recall Curve (PRC)
#plot_prc("Train Predictions", y_train, predictions_train, color=colors[0])
#plot_prc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
#plt.legend(loc='lower right')
#plt.show()

## 1.2 Dense Model with 32 Lines

### 1.2.1 Load Prepared Dataset

In [ ]:
#columns to drop
#columns = ['min', 'max', 'mean', 'orig', 'image', 'algo', 'Image', 'Group']
columns = ['image', 'algo', 'Image', 'Group']

df = df_damage_thirtytwo.drop(columns=columns)

### 1.2.2 Split into sets for training, validation, and testing + use and save scaler

In [ ]:
X = df.drop(columns=['Damage'])
y = df['Damage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=random_state)

scaler_sixteen = StandardScaler()
X_train = scaler_sixteen.fit_transform(X_train)

X_val = scaler_sixteen.transform(X_val)
X_test = scaler_sixteen.transform(X_test)

print('Training features shape:', X_train.shape)
print('Training labels shape:', y_train.shape, '\n')

print('Validation features shape:', X_val.shape)
print('Validation labels shape:', y_val.shape, '\n')

print('Test features shape:', X_test.shape)
print('Test labels shape:', y_test.shape, '\n')

### 1.2.3 Experimentation

#### 1.2.3.1 Configurations

In [ ]:
EPOCHS = 500
BATCH_SIZE = 50

input_dim = X_train.shape[1]
output_size = 1

units = 16
activation = 'relu'
#activation = 'tanh'
kernel_initializer = 'he_uniform'
output_activation = 'sigmoid'
optimizer = 'Adam'
learning_rate = 1e-3

LAYERS = [ 
    { 'type': 'Dense', 'nodes':units, 'activation': activation },
    { 'type': 'Dense', 'nodes':units, 'activation': activation },
    { 'type': 'Dense', 'nodes':output_size, 'activation': output_activation }
]

#### 1.2.3.2 Model

In [ ]:
model = make_dense_model(metrics=METRICS,
                         layers=LAYERS, 
                         loss='binary_crossentropy',
                         optimizer=optimizer,
                         learning_rate=learning_rate, 
                         input_dim=input_dim)

In [ ]:
history = model.fit(x=X_train,
                    y=y_train, 
                    validation_data=(X_val, y_val), 
                    shuffle=True, 
                    epochs=EPOCHS, 
                    verbose=1,
                    batch_size=BATCH_SIZE, 
                    callbacks=[early_stopping])

In [ ]:
early_stopping.stopped_epoch

In [ ]:
#plot_loss(history, "Bias", 0)
#plt.show()

In [ ]:
plot_metrics(history)
#plt.show()

In [ ]:
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)

# Confusion matrix
results = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
    if name == 'accuracy':
        dense_32 = value

In [ ]:
plot_cm(y_test, predictions_test)
#plt.show()

In [ ]:
plot_roc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Precision-Recall Curve (PRC)
#plot_prc("Train Predictions", y_train, predictions_train, color=colors[0])
#plot_prc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
#plt.legend(loc='lower right')
#plt.show()

# 2 CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer

from tensorflow.keras.layers import Conv1D, Dense, MaxPool1D, MaxPooling1D, Dropout, Flatten
from keras.layers import BatchNormalization

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_cnn_model(loss, optimizer, learning_rate, metrics, in_shape, kernel_size, kernel_initializer, pool_size, filters_1, filters_2, filters_3, filters_4):
    
    model = Sequential()
    model.add(Conv1D(filters= filters_1, kernel_size = kernel_size, activation=activation, kernel_initializer=kernel_initializer, input_shape=in_shape))
    
    model.add(Conv1D(filters= filters_2, kernel_size = kernel_size, activation=activation, kernel_initializer=kernel_initializer))
    model.add(MaxPool1D(pool_size=pool_size))
    model.add(Dropout(0.5))
    
    #model.add(Conv1D(filters= filters_3, kernel_size = kernel_size, activation=activation, kernel_initializer=kernel_initializer))
    #model.add(MaxPool1D(pool_size=pool_size))
    #model.add(Dropout(0.5))
    
    model.add(Flatten())
    #model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    #model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(8, activation='relu', kernel_initializer='he_uniform'))
    #model.add(Dropout(0.5))
    #model.add(BatchNormalization())
    model.add(Dense(n_classes, activation=output_activation))
    
    
    optimizer = tf.keras.optimizers.get(optimizer)
    optimizer.learning_rate.assign(learning_rate)

    model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=metrics)

    print(model.summary())

    return model

## 2.1 CNN Model with 16 Lines

### 2.1.1 Load Prepared Dataset

In [ ]:
#columns to drop
columns = ['image', 'algo', 'Image', 'Group']

df = df_damage_sixteen.drop(columns=columns)

### 2.1.2 Split into sets for training, validation, and testing + use and save scaler

In [ ]:
X = df.drop(columns=['Damage'])
y = df['Damage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=random_state)

scaler_sixteen = StandardScaler()
X_train = scaler_sixteen.fit_transform(X_train)

X_val = scaler_sixteen.transform(X_val)
X_test = scaler_sixteen.transform(X_test)

print('Training features shape:', X_train.shape)
print('Training labels shape:', y_train.shape, '\n')

print('Validation features shape:', X_val.shape)
print('Validation labels shape:', y_val.shape, '\n')

print('Test features shape:', X_test.shape)
print('Test labels shape:', y_test.shape, '\n')

### 2.1.3 Experimentation

#### 2.1.3.1 Configurations

In [ ]:
from numpy import unique
orig_shape = X_train.shape[1:]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

in_shape = X_train.shape[1:]
print("Before: {0}".format(orig_shape))
print("After: {0}".format(in_shape))



EPOCHS = 500
BATCH_SIZE = 2050

n_classes = len(unique(y_train))
n_classes = 1
print("Classes: {0}".format(n_classes))

units = 64
activation = 'relu'
kernel_initializer = 'he_uniform'
output_activation = 'sigmoid'
#output_activation = 'softmax'
optimizer = 'Adam'
learning_rate = 1e-3
filters_1 = 8
filters_2 = 16
filters_3 = 32
filters_4 = 12
kernel_size = 3
pool_size = 2
strides = 1

#### 2.1.3.2 Model

In [ ]:
model = make_cnn_model(metrics=METRICS, 
                       loss='binary_crossentropy', 
                       optimizer=optimizer, 
                       learning_rate=learning_rate,
                       in_shape= in_shape,
                       kernel_size=kernel_size,
                       kernel_initializer=kernel_initializer,
                       pool_size=pool_size,
                       filters_1=filters_1,
                       filters_2=filters_2,
                       filters_3=filters_3,
                       filters_4=filters_4)

In [ ]:
history = model.fit(x=X_train,
                    y=y_train, 
                    validation_data=(X_val, y_val), 
                    shuffle=True, 
                    epochs=EPOCHS, 
                    verbose=1,
                    batch_size=BATCH_SIZE, 
                    callbacks=[early_stopping])

In [ ]:
early_stopping.stopped_epoch

In [ ]:
#plot_loss(history, "Bias", 0)
#plt.show()

In [ ]:
plot_metrics(history)
#plt.show()

In [ ]:
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)

# Confusion matrix
results = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
    if name == 'accuracy':
        cnn_16 = value

In [ ]:
plot_cm(y_test, predictions_test)
#plt.show()

In [ ]:
plot_roc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Precision-Recall Curve (PRC)
#plot_prc("Train Predictions", y_train, predictions_train, color=colors[0])
#plot_prc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
#plt.legend(loc='lower right')
#plt.show()

## 2.2 CNN Model with 32 Lines

### 2.2.1 Load Prepared Dataset

In [ ]:
#columns to drop
columns = ['image', 'algo', 'Image', 'Group']

df = df_damage_thirtytwo.drop(columns=columns)

### 2.2.2 Split into sets for training, validation, and testing + use and save scaler

In [ ]:
X = df.drop(columns=['Damage'])
y = df['Damage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=random_state)

scaler_sixteen = StandardScaler()
X_train = scaler_sixteen.fit_transform(X_train)

X_val = scaler_sixteen.transform(X_val)
X_test = scaler_sixteen.transform(X_test)

print('Training features shape:', X_train.shape)
print('Training labels shape:', y_train.shape, '\n')

print('Validation features shape:', X_val.shape)
print('Validation labels shape:', y_val.shape, '\n')

print('Test features shape:', X_test.shape)
print('Test labels shape:', y_test.shape, '\n')

### 2.2.3 Experimentation

#### 2.2.3.1 Configurations

In [ ]:
from numpy import unique
orig_shape = X_train.shape[1:]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

in_shape = X_train.shape[1:]
print("Before: {0}".format(orig_shape))
print("After: {0}".format(in_shape))



EPOCHS = 500
BATCH_SIZE = 2050

n_classes = len(unique(y_train))
n_classes = 1
print("Classes: {0}".format(n_classes))

units = 64
activation = 'relu'
kernel_initializer = 'he_uniform'
output_activation = 'sigmoid'
#output_activation = 'softmax'
optimizer = 'Adam'
learning_rate = 1e-3
filters_1 = 8
filters_2 = 16
filters_3 = 32
filters_4 = 12
kernel_size = 3
pool_size = 2
strides = 1

#### 2.2.3.2 Model

In [ ]:
model = make_cnn_model(metrics=METRICS, 
                       loss='binary_crossentropy', 
                       optimizer=optimizer, 
                       learning_rate=learning_rate,
                       in_shape= in_shape,
                       kernel_size=kernel_size,
                       kernel_initializer=kernel_initializer,
                       pool_size=pool_size,
                       filters_1=filters_1,
                       filters_2=filters_2,
                       filters_3=filters_3,
                       filters_4=filters_4)

In [ ]:
history = model.fit(x=X_train,
                    y=y_train, 
                    validation_data=(X_val, y_val), 
                    shuffle=True, 
                    epochs=EPOCHS, 
                    verbose=1,
                    batch_size=BATCH_SIZE, 
                    callbacks=[early_stopping])

In [ ]:
early_stopping.stopped_epoch

In [ ]:
#plot_loss(history, "Bias", 0)
#plt.show()

In [ ]:
plot_metrics(history)
#plt.show()

In [ ]:
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)

# Confusion matrix
results = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)
    if name == 'accuracy':
        cnn_32 = value

In [ ]:
plot_cm(y_test, predictions_test)
#plt.show()

In [ ]:
plot_roc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Precision-Recall Curve (PRC)
#plot_prc("Train Predictions", y_train, predictions_train, color=colors[0])
#plot_prc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
#plt.legend(loc='lower right')
#plt.show()

# Summarry

In [ ]:
col = [1, 2, 3, 4]
accuracy = [dense_16, dense_32, cnn_16, cnn_32]

tick_label = ['Dense', 'Dense', 'CNN', 'CNN']
plt.bar(col, accuracy, tick_label = tick_label, width = 0.8)
    
plt.xticks(rotation=45, horizontalalignment="right")
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Model vs Accuracy')

plt.show()   